In [28]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 
from pandas import ExcelWriter
from pandas import ExcelFile
import time

In [29]:
def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


In [30]:
def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists



In [31]:
# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0

In [32]:
# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



In [33]:
# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1



In [44]:
# In[7]:

def crawl(original_url, num_Id, output_file):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = []
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()
    Di={}

    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
            
            for child in child_list:
                if child != None:
                    ch=child
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child and ch not in Di:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1 and ch not in Di:
                        child_list_filtered.append(child)
                        Di[ch]=1
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    df.to_csv(output_file + '/' + str(num_Id) + '_' + str(url_to_check) +  '.csv', sep=',', encoding='utf-8')
    return df, num_Id

In [45]:
import requests

page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []

In [46]:
import json
import urllib
import os 
extensions=['.png', '.jpg','jpeg','.pdf', '.css', '.gif','.ico']
def scrape(df, scraped_website_path, website_index, website_name):
    TempD={}
    for index,item in df.iterrows():
        file_save(item[1],item[0])
        try:      
            print(item[1])
            if item.endswith(extensions):
                pass
            html = urllib.request.urlopen(str(item[1])).read()
            #web_text.append(text_from_html(html))
            print(text_from_html(html))
            time.sleep(3)
            #save_url(str(item[1]), index)
            TempD[str(item[1])]=text_from_html(html)
        except: 
            pass
    with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
        json.dump(TempD, outfile)
        
    return 1

In [47]:

# web_crawler_all() = Scraper for multiple websites# web_cr 
'''
Inputs
-----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
web_col = the column no of in which the website hompages are located 
index_col = the column with indices of websites
parent_col = the column with the parent names
output_folder = path of the output folder where we wish to save the websites 
from_website = the index no. from website (typically starting from 1)
to_website = the index no of the website till we want to continue scraping (scraped websites will include the 
                websites with this index)

Output 
-----------
CSV files containing a list of child links for each website are saved into the output_folder path 

'''
def web_crawler_all(excel_path, sheet, web_col, index_col, parent_col,  output_folder, from_website, to_website):
    df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]
    print(df)
    for index,row in df.iterrows():
        num_Id=int(row[index_col])
        original_url=row[web_col]
        print("Website no : " + str(num_Id))
        print("Website name : " + str(original_url))
        
        try:
            websites= pd.read_csv(output_folder + str(int(num_Id)) + '_' + str(original_url) + '.csv')
            print("website is already crawled")
        except:
            websites, num_Id = crawl(original_url ,num_Id ,output_folder)
            
        
    print("All links from the websites indicated have been crawled")
    print("Output File :  " + str(output_folder))
    print("No. of websites scraped : " + str(to_website - from_website + 1))
    return 1

In [ ]:
#executing the crawler function 

excel_path = 'D:\WHIN/trial1.xlsx'
sheet = 'From List of Companies'
web_col = 2
index_col = 0
parent_col = 1
output_folder = 'D:\WHIN\scraped_data\crawled_websites/'
from_website = 1 
to_website = 2 

web_crawler_all(excel_path,sheet, web_col, index_col, parent_col,output_folder, from_website, to_website)

   Serial No.          Company Name                          Website  \
0         2.0    Carmel Engineering        http://www.carmeleng.com/   
1         3.0  Dayton-Phoenix Group  https://www.dayton-phoenix.com/   

   Has a Website?\n(1=Yes, 0=No) Static/Dynamic Website?  \
0                            1.0                       D   
1                            1.0                       D   

   If no website- Is present on Facebook?\n(1=Yes, 0=No)  
0                                                NaN      
1                                                NaN      
Website no : 2
Website name : http://www.carmeleng.com/
www.carmeleng.com
http://www.carmeleng.com/contact_us.asp
http://www.carmeleng.com/newsletter.asp
http://www.carmeleng.com/http:/www.gobuckaroo.com
http://www.carmeleng.com/default.asp
http://www.carmeleng.com//favicon.ico
http://www.carmeleng.com/carmelstyle.css
http://www.carmeleng.com/images/CE_SplashPage-2_03.jpg
http://www.carmeleng.com/images/CE_SplashPage-2_NE

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


No of links = 0


Website no : 3
Website name : https://www.dayton-phoenix.com/
www.dayton-phoenix.com
https://www.dayton-phoenix.com/siteMap.php
https://www.dayton-phoenix.com/siteCredits.php
https://www.dayton-phoenix.com/divisions.php
https://www.dayton-phoenix.com/news.php
https://www.dayton-phoenix.com/productCategories.php
https://www.dayton-phoenix.com/serviceCenters.php
https://www.dayton-phoenix.com/contact.php
https://www.dayton-phoenix.com//divisions.php
https://www.dayton-phoenix.com/index.php?dt=FL
https://www.dayton-phoenix.com/index.php?dt=PL
https://www.dayton-phoenix.com/index.php?dt=OH
https://www.dayton-phoenix.com/index.php?dt=ES
https://www.dayton-phoenix.com/index.php?dt=SV
https://www.dayton-phoenix.com/news.php?newsId=38
https://www.dayton-phoenix.com/news.php?newsId=37
https://www.dayton-phoenix.com/news.php?newsId=36
https://www.dayton-phoenix.com/news.php?newsId=35
https://www.dayton-phoenix.com/productCategoryDetail.php?categoryId=106
https://www.dayton-phoe

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.dayton-phoenix.com/productCategories.php?categoryId=115
https://www.dayton-phoenix.com/productCategories.php?categoryId=54
https://www.dayton-phoenix.com/productCategories.php?categoryId=55
https://www.dayton-phoenix.com/productCategories.php?categoryId=21
https://www.dayton-phoenix.com/productCategories.php?categoryId=92
https://www.dayton-phoenix.com/productCategories.php?categoryId=58
https://www.dayton-phoenix.com/productCategories.php?categoryId=122
https://www.dayton-phoenix.com/productCategories.php?categoryId=124
https://www.dayton-phoenix.com/productCategories.php?categoryId=116
https://www.dayton-phoenix.com/news.php?newsId=34
https://www.dayton-phoenix.com/news.php?newsId=27
https://www.dayton-phoenix.com/news.php?newsId=26
https://www.dayton-phoenix.com/minerals.php
https://www.dayton-phoenix.com/news.php?newsId=28
https://www.dayton-phoenix.com/suppdiv.php
https://www.dayton-phoenix.com/productCategories.php?categoryId=59
https://www.dayton-phoenix.com/productC

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

https://www.dayton-phoenix.com/news.php?newsId=27/documents/uploads/Press Release DPG Grid Resistor Quality .pdf
https://www.dayton-phoenix.com/news.php?newsId=26/documents/uploads/Press Release DPG Announces Nick Frazier.pdf
https://www.dayton-phoenix.com/news.php?newsId=28/documents/uploads/Press Release DPG Receives TPG Accrediation .pdf
https://www.dayton-phoenix.com/news.php?newsId=33/documents/uploads/Dayton-Phoenix Group Announces new Advantage SeriesTM Cab Heater and Phase Out of Older PM Series Cab Heater.pdf
https://www.dayton-phoenix.com/news.php?newsId=32/documents/uploads/Press Release DPG Announces Advantage Series 42517.pdf
https://www.dayton-phoenix.com/dynamic_supplier_manual/#Nav_General Description
https://www.dayton-phoenix.com/dynamic_supplier_manual/#Nav_Requirements
https://www.dayton-phoenix.com/dynamic_supplier_manual/#Nav_Detailed StepsOverview of Steps
https://www.dayton-phoenix.com/dynamic_supplier_manual/#Nav_Detailed Steps
https://www.dayton-phoenix.com/dy